# **Setup**

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('preprocessed_spam_ham_phishing.csv')

**GET PHISHING/HAM EMAILS**

In [3]:
# Remove spam emails, only consider phishing: (spam = 1, ham = 0, phishing = 2)
df_phish = df[(df['label'] == 2)]
print(df_phish.shape)

(1288, 95)


In [4]:
# Ham data
df_ham = df[(df['label'] == 0)]
df_ham.shape

(25220, 95)

In [5]:
# Split ham data into 50/50 so some can be used for validation
df_split = df_ham[:int(len(df_ham)/2)]
df_split = df_split.reset_index()
df_split = df_split.drop('index', axis=1)

df_val_ham = df_ham[int(len(df_ham)/2):]
df_val_ham = df_val_ham.reset_index()
df_val_ham = df_val_ham.drop('index', axis=1)

df = pd.concat([df_phish, df_split])
print(df.shape)

(13898, 95)


In [6]:
df['label'].value_counts()

label
0    12610
2     1288
Name: count, dtype: int64

In [7]:
# Randomly Sample 1288 Ham emails to create a balanced dataset
df_ham = df[df['label'] == 0].sample(1288)
df_phish = df[df['label'] == 2]

df_phish = df_phish.assign(label=1)

df_new = df_ham._append(df_phish, ignore_index=True)
df_new = df_new.sample(frac=1)
df = df_new.reset_index(drop=True)
df

,hops,missing_subject,missing_to,missing_content-type,missing_mime-version,missing_x-mailer,missing_content-transfer-encoding,missing_x-mimeole,missing_x-priority,missing_list-id,...,domain_match_errors-to_reply-to,domain_match_sender_from,domain_match_references_reply-to,domain_match_references_in-reply-to,domain_match_references_to,domain_match_from_reply-to,domain_match_to_from,domain_match_to_message-id,domain_match_to_received,label
0,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,1,1,0,1,1,0,...,1,1,0,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,1,0,0,0,0,1,0,1,1,0,...,0,1,0,1,1,0,0,0,0,0
2572,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,1,0,0,0,1
2573,2,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2574,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,1,1,0,0,1


In [8]:
# Reduce feature set
# The only features that are kept are domain matching features, as these should generalize across very different email datasets without issue.

feature_list = [
'domain_match_from_return-path',
'domain_match_message-id_from',
'domain_match_message-id_return-path',
'domain_match_to_from',
'domain_match_errors-to_from',
'domain_match_message-id_reply-to',
'domain_match_errors-to_message-id',
'domain_match_sender_from',
'domain_match_to_received',
'domain_match_errors-to_reply-to',
'domain_match_to_message-id',
'label']

feature_list = ['domain_val_message-id',
       'domain_match_message-id_from', 'domain_match_from_return-path',
       'domain_match_message-id_return-path', 'domain_match_message-id_sender',
       'domain_match_message-id_reply-to', 'domain_match_return-path_reply-to',
       'domain_match_reply-to_to', 'domain_match_to_in-reply-to',
       'domain_match_errors-to_message-id', 'domain_match_errors-to_from',
       'domain_match_errors-to_sender', 'domain_match_errors-to_reply-to',
       'domain_match_sender_from', 'domain_match_references_reply-to',
       'domain_match_references_in-reply-to', 'domain_match_references_to',
       'domain_match_from_reply-to', 'domain_match_to_from',
       'domain_match_to_message-id', 'domain_match_to_received', 'label']

df = df[feature_list]

In [9]:
df_Y = df['label']
df_X = df.drop('label', axis=1)

features_list = df_X.columns

In [10]:
# Apply a standard scaler to the full data set
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(df_X)
df_X = scaler.transform(df_X)
df_X = pd.DataFrame(df_X, columns=features_list)

In [11]:
# Breaking the data into a test and training set (20% test, 80% train)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.20, random_state=42)

**GET VALIDATION DATA**

In [12]:
# This is needed in order to have ham data
df_val = pd.read_csv('preprocessed_phishing_2022.csv')

In [13]:
# The solution to get ham data is to transfer from other data...

df_val = pd.concat([df_val, df_val_ham])

In [14]:
df_val['label'].value_counts()

label
0    12610
2      245
Name: count, dtype: int64

In [15]:
# Randomly Sample 245 Ham emails to create a balanced dataset
df_ham_val = df_val[df_val['label'] == 0].sample(245)

df_phish_val = df_val[df_val['label'] == 2]

df_phish_val = df_phish_val.assign(label=1)

df_new_val = df_ham_val._append(df_phish_val, ignore_index=True)

df_new_val = df_new_val.sample(frac=1)
df_val = df_new_val.reset_index(drop=True)

In [16]:
# Reduce feature set
# The only features that are kept are domain matching features, as these should generalize across very different email datasets without issue.

feature_list = [
'domain_match_from_return-path',
'domain_match_message-id_from',
'domain_match_message-id_return-path',
'domain_match_to_from',
'domain_match_errors-to_from',
'domain_match_message-id_reply-to',
'domain_match_errors-to_message-id',
'domain_match_sender_from',
'domain_match_to_received',
'domain_match_errors-to_reply-to',
'domain_match_to_message-id',
'label']

feature_list = ['domain_val_message-id',
       'domain_match_message-id_from', 'domain_match_from_return-path',
       'domain_match_message-id_return-path', 'domain_match_message-id_sender',
       'domain_match_message-id_reply-to', 'domain_match_return-path_reply-to',
       'domain_match_reply-to_to', 'domain_match_to_in-reply-to',
       'domain_match_errors-to_message-id', 'domain_match_errors-to_from',
       'domain_match_errors-to_sender', 'domain_match_errors-to_reply-to',
       'domain_match_sender_from', 'domain_match_references_reply-to',
       'domain_match_references_in-reply-to', 'domain_match_references_to',
       'domain_match_from_reply-to', 'domain_match_to_from',
       'domain_match_to_message-id', 'domain_match_to_received', 'label']

df_val = df_val[feature_list]

X_val = df_val.drop('label', axis=1)
y_val = df_val['label']

# **Random Forest**

In [17]:
%%time
### Testing with hyperparameter tuning
pipe = Pipeline([("scale", StandardScaler()),
                ("rf", RandomForestClassifier())
                ])

param_grid_list = {'rf__n_estimators': [100, 150],
                  'rf__criterion': ['entropy', 'gini'],
                  'rf__min_samples_split': [2, 3],
                  'rf__min_samples_leaf': [1, 2],
                  'rf__max_features': ['auto', 'sqrt', 'log2']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

#Check the hyperparameter results
rf_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

#Get the best performing model
best_model = grid.best_estimator_

original_predict = best_model.predict(X_test)

# # Test the best performing model on the test set

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, original_predict)*100)
print('F1 Score:', f1_score(y_test, original_predict)*100)
print('Recall:', recall_score(y_test, original_predict)*100)
print('Precision:', precision_score(y_test, original_predict)*100)
print('ROC AUC:', roc_auc_score(y_test, original_predict)*100)
print('Confusion Matrix:', confusion_matrix(y_test, original_predict))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

rf_df[rf_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 48 candidates, totalling 480 fits
[CV 1/10; 1/48] START rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 1/10; 1/48] END rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=nan total time=   0.0s
[CV 1/10; 2/48] START rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=150
[CV 1/10; 2/48] END rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=150;, score=nan total time=   0.0s
[CV 2/10; 2/48] START rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=150
[CV 2/10; 2/48] END rf__criterion=entropy, rf__max_features=auto, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=150;, score=nan total time=   0.0s
[CV 3/10; 2/48] START 

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
160 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 427, in f

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__criterion,param_rf__max_features,param_rf__min_samples_leaf,param_rf__min_samples_split,param_rf__n_estimators,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
8,0.099607,0.019746,0.005251,0.004425,entropy,sqrt,1,2,100,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.966019,0.980583,0.961165,0.966019,0.975728,0.990291,0.976214,0.009817,2
9,0.129424,0.018845,0.005929,0.001548,entropy,sqrt,1,2,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.966019,0.980583,0.961165,0.966019,0.975728,0.990291,0.976214,0.009817,2
10,0.094261,0.025129,0.007637,0.005202,entropy,sqrt,1,3,100,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.966019,0.980583,0.961165,0.966019,0.975728,0.990291,0.976214,0.009817,2
11,0.143379,0.016561,0.007758,0.003994,entropy,sqrt,1,3,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.966019,0.980583,0.961165,0.966019,0.975728,0.990291,0.976214,0.009817,2
19,0.125749,0.007061,0.006757,0.001833,entropy,log2,1,3,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.966019,0.980583,0.961165,0.966019,0.975728,0.990291,0.976214,0.009817,2


# **MLP**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("mlp", MLPClassifier())
                ])

param_grid_list = {'mlp__hidden_layer_sizes': [(20,), (20,20), (40,), (40,40)],
                   'mlp__activation': ['tanh', 'relu'],
                   'mlp__learning_rate': ['constant', 'adaptive'],
                   'mlp__solver': ['adam', 'sgd'],
                   'mlp__alpha': [0.0001, 0.001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
mlp_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

mlp_df[mlp_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
[CV 1/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 7/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 3/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 4/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 9/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 10/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 2/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), ml

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 8/10; 2/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd
[CV 9/10; 2/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd
[CV 10/10; 2/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd
[CV 1/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.981 total time=   0.3s
[CV 1/10; 3/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=adam


/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

[CV 2/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.976 total time=   0.3s
[CV 2/10; 3/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 5/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.981 total time=   0.3s
[CV 3/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.990 total time=   0.3s
[CV 3/10; 3/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 4/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.932 total time=   0.3s
[CV 4/10; 3/96] START mlp__activation=ta

...

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 10/10; 93/96] START mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=adam
[CV 1/10; 93/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=adam;, score=0.985 total time=   0.3s
[CV 1/10; 94/96] START mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=sgd
[CV 2/10; 93/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=adam;, score=0.971 total time=   0.3s
[CV 2/10; 94/96] START mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=sgd
[CV 5/10; 93/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=adam;, score=0.985 total time=   0.3s
[CV 3/10; 94/96] START mlp__activation=relu, mlp__alpha=

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

[CV 3/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.990 total time=   0.6s
[CV 2/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.971 total time=   0.6s
[CV 5/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.981 total time=   0.6s
[CV 6/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.976 total time=   0.6s
[CV 4/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.937 total time=   0.7s
[CV 7/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;,

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__solver,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
53,0.399122,0.026856,0.000939,0.000380,relu,0.0001,"(20, 20)",constant,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.941748,0.980583,0.980583,0.966019,0.985437,0.970874,0.995146,0.977184,0.014732,1
59,0.360633,0.042084,0.001248,0.000946,relu,0.0001,"(40,)",adaptive,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.936893,0.985437,0.975728,0.966019,0.985437,0.970874,0.995146,0.976699,0.016071,3
77,0.588416,0.070824,0.001071,0.000649,relu,0.001,"(40, 40)",constant,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.932039,0.985437,0.980583,0.966019,0.985437,0.966019,0.995146,0.976699,0.017743,3
78,0.309040,0.046090,0.001152,0.000605,relu,0.001,"(40, 40)",adaptive,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.936893,0.985437,0.980583,0.961165,0.985437,0.966019,0.995146,0.976699,0.016928,3
79,0.646383,0.032603,0.001451,0.001110,relu,0.001,"(40, 40)",adaptive,sgd,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.936893,0.980583,0.985437,0.966019,0.985437,0.966019,0.995146,0.977184,0.016541,1


# **Logistic Regression**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("lr", LogisticRegression())
                ])

param_grid_list = {'lr__max_iter': [500],
                  'lr__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                  'lr__fit_intercept': [True, False],
                  'lr__tol': [0.0001, 0.001],
                  'lr__penalty': ['l1', 'l2', 'elasticnet'],
                  'lr__C': [0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
lr_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

lr_df[lr_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[CV 3/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 2/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 4/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 1/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 5/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 3/10; 1/144] END lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001;, score=nan total time=   0.0s
[CV 7/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/10; 109/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=sag, lr__tol=0.0001;, score=0.985 total time=   0.1s
[CV 2/10; 109/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=sag, lr__tol=0.0001
[CV 2/10; 64/144] END lr__C=1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.001;, score=0.976 total time=   0.0s
[CV 3/10; 64/144] START lr__C=1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.001
[CV 2/10; 87/144] END lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001;, score=0.976 total time=   0.1s
[CV 3/10; 87/144] START lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001
[CV 3/10; 64/144] END lr__C=1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.001;, score=0.995 total time=   0.0s
[CV 4/10; 64/1

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/10; 111/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001;, score=0.985 total time=   0.1s
[CV 6/10; 111/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001
[CV 9/10; 87/144] END lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001;, score=0.966 total time=   0.1s
[CV 10/10; 87/144] START lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001
[CV 10/10; 79/144] END lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.995 total time=   0.1s
[CV 1/10; 80/144] START lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 7/10; 109/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=sag, lr__tol=0.0001;, score=0.966 total time=   0.1s
[CV 8

...

[CV 9/10; 103/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.961 total time=   0.1s
[CV 10/10; 103/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001
[CV 10/10; 103/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.990 total time=   0.1s
[CV 1/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 1/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.985 total time=   0.0s
[CV 2/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001


/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.976 total time=   0.0s
[CV 3/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 3/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.995 total time=   0.0s
[CV 4/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 4/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.937 total time=   0.0s
[CV 5/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 5/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.985 total time=   0.0s
[CV 6/10

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
840 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 427, in

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr__C,param_lr__fit_intercept,param_lr__max_iter,param_lr__penalty,param_lr__solver,param_lr__tol,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
54,0.120726,0.023974,0.000984,0.000398,1,True,500,l1,saga,0.0001,...,0.936893,0.985437,0.980583,0.966019,0.985437,0.966019,0.995146,0.977184,0.016541,1
55,0.041354,0.011879,0.001055,0.000460,1,True,500,l1,saga,0.001,...,0.936893,0.985437,0.975728,0.966019,0.985437,0.966019,0.995146,0.976699,0.016505,2
56,0.012564,0.004538,0.000963,0.000551,1,True,500,l2,newton-cg,0.0001,...,0.936893,0.985437,0.975728,0.966019,0.985437,0.961165,0.995146,0.976214,0.016879,3
57,0.008793,0.004098,0.001138,0.000765,1,True,500,l2,newton-cg,0.001,...,0.936893,0.985437,0.975728,0.966019,0.985437,0.961165,0.995146,0.976214,0.016879,3
58,0.006283,0.001336,0.000907,0.000480,1,True,500,l2,lbfgs,0.0001,...,0.936893,0.985437,0.975728,0.966019,0.985437,0.961165,0.995146,0.976214,0.016879,3


# **SVM**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("svc", SVC())
                ])

param_grid_list = {'svc__C': [0.1, 1, 10],
                  'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                  'svc__degree': [3, 4, 5],
                  'svc__tol': [0.001, 0.0001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
svm_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

svm_df[svm_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV 1/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 2/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 3/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 1/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.966 total time=   0.0s
[CV 4/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 2/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.971 total time=   0.0s
[CV 5/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 3/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.990 total time=   0.0s
[CV 6/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 7/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__to

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__degree,param_svc__kernel,param_svc__tol,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
42,0.010125,0.001570,0.002592,0.000844,1,3,rbf,0.001,"{'svc__C': 1, 'svc__degree': 3, 'svc__kernel':...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.970874,0.995146,0.977184,0.016682,1
43,0.010437,0.002325,0.003143,0.001197,1,3,rbf,0.0001,"{'svc__C': 1, 'svc__degree': 3, 'svc__kernel':...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.970874,0.995146,0.977184,0.016682,1
44,0.010183,0.002933,0.003422,0.002188,1,3,rbf,0.01,"{'svc__C': 1, 'svc__degree': 3, 'svc__kernel':...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.970874,0.995146,0.977184,0.016682,1
54,0.013095,0.003753,0.003037,0.001425,1,4,rbf,0.001,"{'svc__C': 1, 'svc__degree': 4, 'svc__kernel':...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.970874,0.995146,0.977184,0.016682,1
55,0.010417,0.003266,0.002451,0.000709,1,4,rbf,0.0001,"{'svc__C': 1, 'svc__degree': 4, 'svc__kernel':...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.970874,0.995146,0.977184,0.016682,1


# **Decision Tree**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("dt", DecisionTreeClassifier())
                ])

param_grid_list = {'dt__criterion': ['entropy', 'gini'],
                  'dt__min_samples_split': [2, 3, 4],
                  'dt__min_samples_leaf': [1, 2, 3],
                  'dt__ccp_alpha': [0, 0.005, 0.01, 0.025, 0.05, 0.1]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
dt_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

dt_df[dt_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV 4/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 2/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 4/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.937 total time=   0.0s
[CV 1/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 2/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.971 total time=   0.0s
[CV 1/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.985 total time=   0.0s
[CV 10/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 3/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, d

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dt__ccp_alpha,param_dt__criterion,param_dt__min_samples_leaf,param_dt__min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002525,0.000651,0.000785,0.000393,0,entropy,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.975728,0.961165,0.995146,0.974272,0.016541,1
1,0.002937,0.001206,0.000662,0.000053,0,entropy,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.985437,...,0.932039,0.985437,0.980583,0.961165,0.975728,0.961165,0.995146,0.973786,0.017664,4
2,0.002226,0.000550,0.000601,0.000029,0,entropy,1,4,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.985437,...,0.932039,0.985437,0.980583,0.961165,0.975728,0.961165,0.995146,0.973786,0.017664,4
9,0.001932,0.000280,0.000839,0.000541,0,gini,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.975728,0.961165,0.995146,0.974272,0.016541,1
10,0.001952,0.000296,0.000766,0.000524,0,gini,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.985437,...,0.936893,0.985437,0.980583,0.961165,0.975728,0.961165,0.995146,0.974272,0.016541,1


# **Naive Bayes (Gaussian)**

In [17]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gnb", GaussianNB())
                ])

param_grid_list = {'gnb__var_smoothing': [1E-9, 1E-10, 1E-8]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
nb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

nb_df[nb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV 2/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 2/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.903 total time=   0.0s
[CV 1/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 1/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.869 total time=   0.0s
[CV 1/10; 2/3] START gnb__var_smoothing=1e-10...................................
[CV 2/10; 2/3] START gnb__var_smoothing=1e-10...................................
[CV 1/10; 2/3] END ....gnb__var_smoothing=1e-10;, score=0.869 total time=   0.0s
[CV 2/10; 2/3] END ....gnb__var_smoothing=1e-10;, score=0.903 total time=   0.0s
[CV 3/10; 2/3] START gnb__var_smoothing=1e-10...................................
[CV 4/10; 2/3] START gnb__var_smoothing=1e-10...................................
[CV 3/10; 2/3] END ....gnb__var_smoothing=1e-10;, score=0.903 total time=   0.0s
[CV 4/10; 2/3] END ....gnb__var_smoothing=1e-10;

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gnb__var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002713,0.001054,0.000875,0.000429,0.0,{'gnb__var_smoothing': 1e-09},0.868932,0.902913,0.902913,0.912621,0.864078,0.888350,0.907767,0.873786,0.883495,0.893204,0.889806,0.016107,2
1,0.003718,0.001563,0.001324,0.000967,0.0,{'gnb__var_smoothing': 1e-10},0.868932,0.902913,0.902913,0.912621,0.868932,0.888350,0.907767,0.873786,0.878641,0.893204,0.889806,0.015662,2
2,0.003763,0.001424,0.002218,0.002352,0.0,{'gnb__var_smoothing': 1e-08},0.868932,0.907767,0.912621,0.902913,0.873786,0.878641,0.912621,0.873786,0.883495,0.898058,0.891262,0.016419,1


# **AdaBoost**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("ab", AdaBoostClassifier())
                ])

param_grid_list = {'ab__n_estimators': [50, 100, 150, 200],
                  'ab__learning_rate': [0.95, 1, 1.05, 1.25, 1.5, 1.75, 2],
                  'ab__algorithm': ['SAMME', 'SAMME.R']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
ab_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

ab_df[ab_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 56 candidates, totalling 560 fits
[CV 1/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 2/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 4/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 5/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50[CV 3/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50

[CV 7/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 6/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 9/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 8/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 3/10; 1/56] END ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50;, score=0.995 total time=   0.0s
[CV 5/10; 1/56] END ab__algorithm=SA

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ab__algorithm,param_ab__learning_rate,param_ab__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
19,0.239497,0.064034,0.015408,0.005883,SAMME,1.5,200,"{'ab__algorithm': 'SAMME', 'ab__learning_rate'...",0.985437,0.970874,...,0.936893,0.985437,0.970874,0.966019,0.975728,0.961165,0.995146,0.974272,0.016682,1
30,0.134487,0.006273,0.009200,0.001448,SAMME.R,0.95,150,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.985437,0.970874,...,0.936893,0.985437,0.970874,0.966019,0.980583,0.956311,0.990291,0.973786,0.016704,2
31,0.247837,0.067853,0.018522,0.010893,SAMME.R,0.95,200,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.985437,0.970874,...,0.936893,0.985437,0.970874,0.966019,0.980583,0.956311,0.990291,0.973786,0.016704,2
35,0.224061,0.042918,0.015777,0.007625,SAMME.R,1,200,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.985437,0.970874,...,0.936893,0.985437,0.970874,0.966019,0.980583,0.956311,0.990291,0.973786,0.016704,2
37,0.121704,0.027279,0.008286,0.004624,SAMME.R,1.05,100,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.985437,0.970874,...,0.936893,0.985437,0.970874,0.966019,0.980583,0.956311,0.990291,0.973786,0.016704,2


# **GradientBoostClassifier**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gbc", GradientBoostingClassifier())
                ])

param_grid_list = {'gbc__max_features': ['auto', 'sqrt', 'log2'],
                   'gbc__learning_rate': [0.05, 0.1, 0.2, 0.25, 0.30, 0.35, 0.40, 0.5, 0.6, 0.7, 0.9],
                   'gbc__n_estimators': [100, 200]
                  }

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
gb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

gb_df[gb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 66 candidates, totalling 660 fits
[CV 1/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 2/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 1/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 3/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 4/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 2/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 4/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 3/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 5/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
220 fits failed out of a total of 660.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
143 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 427, in 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gbc__learning_rate,param_gbc__max_features,param_gbc__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
3,0.118715,0.030325,0.002771,0.002172,0.05,sqrt,200,"{'gbc__learning_rate': 0.05, 'gbc__max_feature...",0.985437,0.975728,...,0.936893,0.985437,0.980583,0.966019,0.985437,0.966019,0.995146,0.977184,0.016541,1
5,0.087859,0.003362,0.001358,0.000171,0.05,log2,200,"{'gbc__learning_rate': 0.05, 'gbc__max_feature...",0.985437,0.975728,...,0.936893,0.980583,0.980583,0.966019,0.985437,0.966019,0.995146,0.976699,0.016361,4
8,0.077724,0.010824,0.002028,0.001019,0.1,sqrt,100,"{'gbc__learning_rate': 0.1, 'gbc__max_features...",0.985437,0.975728,...,0.936893,0.980583,0.980583,0.966019,0.985437,0.966019,0.995146,0.976699,0.016361,4
9,0.123153,0.022498,0.001477,0.000551,0.1,sqrt,200,"{'gbc__learning_rate': 0.1, 'gbc__max_features...",0.985437,0.975728,...,0.936893,0.985437,0.980583,0.961165,0.985437,0.966019,0.995146,0.976699,0.016928,4
10,0.052316,0.004914,0.001035,0.000026,0.1,log2,100,"{'gbc__learning_rate': 0.1, 'gbc__max_features...",0.980583,0.975728,...,0.936893,0.985437,0.985437,0.966019,0.985437,0.966019,0.995146,0.977184,0.016541,1


# **KNN**

In [ ]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("knn", KNeighborsClassifier())
                ])

param_grid_list = {'knn__n_neighbors': [1, 10, 20],
                  'knn__weights': ['uniform', 'distance'],
                  'knn__p': [1, 2],
                  'knn__algorithm': ['auto'],
                  'knn__leaf_size': [15, 30, 45]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
knn_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

### Validation
val_pred = best_model.predict(X_val)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

# Get the validation evaluation metrics
print('Accuracy:', accuracy_score(y_val, val_pred)*100)
print('F1 Score:', f1_score(y_val, val_pred)*100)
print('Recall:', recall_score(y_val, val_pred)*100)
print('Precision:', precision_score(y_val, val_pred)*100)
print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

knn_df[knn_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[CV 1/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 2/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 3/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 4/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 5/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 6/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 7/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 8/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 9/10; 1/36] START knn__algori

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__algorithm,param_knn__leaf_size,param_knn__n_neighbors,param_knn__p,param_knn__weights,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
5,0.001870,0.000987,0.007108,0.003485,auto,15,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.936893,0.985437,0.985437,0.961165,0.975728,0.961165,0.995146,0.974757,0.016788,4
7,0.002361,0.001819,0.001994,0.000272,auto,15,10,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.936893,0.985437,0.985437,0.961165,0.970874,0.966019,0.995146,0.974757,0.016505,1
17,0.001660,0.000725,0.006448,0.001744,auto,30,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.936893,0.985437,0.985437,0.961165,0.975728,0.961165,0.995146,0.974757,0.016788,4
19,0.001694,0.001067,0.003054,0.001759,auto,30,10,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.936893,0.985437,0.985437,0.961165,0.970874,0.966019,0.995146,0.974757,0.016505,1
29,0.002335,0.002285,0.005960,0.001868,auto,45,10,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 4...",...,0.936893,0.985437,0.985437,0.961165,0.975728,0.961165,0.995146,0.974757,0.016788,4


# **Stacked Testing:**

In [ ]:
from sklearn.ensemble import StackingClassifier

'''
base_learners = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]
'''

base_learners_set1 = [('rf', RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance'))]

base_learners_set2 = [('rf', RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set3 = [('rf', RandomForestClassifier(criterion='entropy',  min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set4 = [('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners = []
base_learners.append(base_learners_set1)
base_learners.append(base_learners_set2)
base_learners.append(base_learners_set3)
base_learners.append(base_learners_set4)

for base_learner_group in base_learners:

    meta_learner = LogisticRegression()

    clf = StackingClassifier(estimators=base_learner_group, final_estimator=meta_learner)

    # Train the stacked model on the full training data
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    ### Validation
    val_pred = clf.predict(X_val)

    # Get the evaluation metrics
    print('Accuracy:', accuracy_score(y_test, predictions)*100)
    print('F1 Score:', f1_score(y_test, predictions)*100)
    print('Recall:', recall_score(y_test, predictions)*100)
    print('Precision:', precision_score(y_test, predictions)*100)
    print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
    print('Confusion Matrix:', confusion_matrix(y_test, predictions))
    print('-----------------------------------------\n')

    # Get the validation evaluation metrics
    print('Accuracy:', accuracy_score(y_val, val_pred)*100)
    print('F1 Score:', f1_score(y_val, val_pred)*100)
    print('Recall:', recall_score(y_val, val_pred)*100)
    print('Precision:', precision_score(y_val, val_pred)*100)
    print('ROC AUC:', roc_auc_score(y_val, val_pred)*100)
    print('Confusion Matrix:', confusion_matrix(y_val, val_pred))

    

Accuracy: 97.86821705426357
F1 Score: 97.85575048732942
Recall: 98.81889763779527
Precision: 96.91119691119691
ROC AUC: 97.88273126164572
Confusion Matrix: [[254   8]
 [  3 251]]
-----------------------------------------

Accuracy: 84.08163265306122
F1 Score: 85.66176470588235
Recall: 95.10204081632652
Precision: 77.92642140468227
ROC AUC: 84.08163265306122
Confusion Matrix: [[179  66]
 [ 12 233]]
Accuracy: 97.86821705426357
F1 Score: 97.85575048732942
Recall: 98.81889763779527
Precision: 96.91119691119691
ROC AUC: 97.88273126164572
Confusion Matrix: [[254   8]
 [  3 251]]
-----------------------------------------

Accuracy: 85.91836734693878
F1 Score: 87.10280373831776
Recall: 95.10204081632652
Precision: 80.3448275862069
ROC AUC: 85.91836734693878
Confusion Matrix: [[188  57]
 [ 12 233]]
Accuracy: 97.86821705426357
F1 Score: 97.85575048732942
Recall: 98.81889763779527
Precision: 96.91119691119691
ROC AUC: 97.88273126164572
Confusion Matrix: [[254   8]
 [  3 251]]
--------------------